# Sentiment Analysis Web App

[Vedant Dave](https://vedantdave77.github.io/) | Vedantdave77@gmail.com | [LinkedIn](https://www.linkedin.com/in/vedant-dave117/)

Hello, I am Vedant Dave, a machine learning practitioner data enthusiast professional. -@dave117

---

In this notebook, I am going to analyze IMDB dataset. Its one of the best dagtaset of NLP research. You can search about IMDB on IMDB.com to get an idea about the company portfolio and their workprofile. Well, my main purpose is to deploy web app using lambda function.

Before this I already worked on Batch_transform job using python SDK, sagemaker hyperparameter tuning and model updation using new generated data. 

In this notebook I will use Amazon's SageMaker service to construct a random tree model to predict the sentiment of a movie review. In addition, we will deploy this model to an endpoint and construct a very simple web app which will interact with our model's deployed endpoint.

## General Outline

The flow of process is as follow:

1. Download or otherwise retrieve the data. 
2. Process / Prepare the data.................................[Data Preprocessing]
3. Upload the processed data to S3..........................[DataBase creation] ---------- AWS-S3
4. Train a chosen model.............................................[Development-Training Phase] ----------- AWS-SageMaker
5. Test the trained model (a batch transform job)........[Testing Phase]
6. Deploy the trained model...........................................[Model Release] ------------ AWS-Lambda
7. Use the deployed model..............................................[Model Running Test]------------WEB_API

In this notebook we will progress through each of the steps above. We will also see that the final step, using the deployed model, can be quite challenging.

---

## Download data from [IMDB dataset](http://ai.stanford.edu/~amaas/data/sentiment/)
Current format of data is One file, for project we need to seperate them in train, validation and test datasets. The labels are also in pos/ neg form so, for project, its better to covert them in 0 and 1

In [1]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

--2020-06-14 18:51:43--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  46.3MB/s    in 1.7s    

2020-06-14 18:51:45 (46.3 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Data Preparation

The complex problem in ML is to clean data, and make them ready for analysis. Here, please observe above review. We downloaded from web in html form. That's why you can see html format <br> ... </br> there. So, first we need to remove them. More over, some words are repetative, meaning less and with similar meaning. So, first we will remove all these obsecles. The step is called data preprocessing, also know as data cleaning, dfata wrangling, data manipulation. So, I am going to use NLTK library.


In [2]:
import os                                                                       # provide operating system accordingly ...
import glob                                                                     # glob is path name matcher, start each file with .*

def read_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [6]:
data, labels = read_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(labels['test']['pos']), len(labels['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


In [7]:
# Now, lets conmbine pos and neg dataset and shuffle them for making training and testing dataset.
# WHY?  --> because, form above function we get four sets separated by pos, neg in train and test set... (look and understand)
from sklearn.utils import shuffle

def prepare_imdb_data(data,lables):
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']              # Awesome mistake +++ cost me 8 days (and 50+ hr sagemaker cost)
    labels_test = labels['test']['pos'] + labels['test']['neg']

    # shuffle reviews and correspoing labels within training and test dataset
    data_train, labels_train = shuffle(data_train,labels_train)                 # this helps us to shuffle through whole training ...
    data_test, labels_test = shuffle(data_test,labels_test)

    # return a datasets for future processes.
    return data_train, data_test, labels_train, labels_test

In [8]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


In [9]:
train_X[100]

'this was a very good movie i wished i could find it in vhs to buy,i really enjoyed this movie i would definaetly recommend this movie to watch i would like to see it again but can never find it in tv, it would be well worth the time to watch it again'

## Data Preprocessing
The complex problem in ML is to clean data, and make them ready for analysis. Here, please observe above review. We downloaded from web in html form. That's why you can see html format **(<!br>  \</!br>)** there. So, first we need to remove them. More over, some words are repetative, meaning less and with similar meaning. So, first we will remove all these obsecles. The step is called data preprocessing, also know as data cleaning, dfata wrangling, data manipulation. So, I am going to use NLTK library.

In [10]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def review_to_words(review):
    words = REPLACE_NO_SPACE.sub("", review.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    return words

In [11]:
review_to_words(train_X[100])

'this was a very good movie i wished i could find it in vhs to buyi really enjoyed this movie i would definaetly recommend this movie to watch i would like to see it again but can never find it in tv it would be well worth the time to watch it again'

In [12]:
import pickle                                                                   # for serializing/deserializing python input (here,pickle--> converts datastructure to byte stram)
cache_dir = os.path.join("../cache", "sentiment_analysis")                      # define storage path
os.makedirs(cache_dir, exist_ok=True)                                           # ensure about directory

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir = cache_dir, cache_file ="preprocessed_data.pkl"):
  
    cache_data = None                                          # initialize cach data
    if cache_file is not None:                                 # comp saved cache data for future purpose so, the operation will be faster 
        try: 
            with open(os.path.join(cache_dir, cache_file), "rb") as f:           # read bite form pickle file
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file :" , cache_file)
        except:
            pass                       

    if cache_data is None:
        words_train = [review_to_words(review) for review in data_train]    # generate list [] from available dict. "data_train"
        words_test = [review_to_words(review) for review in data_test]     # ... same 

        if cache_file is not None:
            cache_data = dict(words_train = words_train,words_test = words_test, labels_train = labels_train,labels_test=labels_test)
        with open(os.path.join(cache_dir, cache_file),"wb") as f:
            pickle.dump(cache_data,f)
            print("Wrote preprocessed data to cache file: ", cache_file)
    else: 
        print("Getting from cache data ...")
        words_train,words_test,labels_train,labels_test = (cache_data['words_train'],cache_data['words_test'],cache_data['labels_train'],cache_data['labels_test'])
      
    return words_train,words_test,labels_train,labels_test

In [13]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Wrote preprocessed data to cache file:  preprocessed_data.pkl


### **Explainations of preprocess operation:** 

Here, We had two options, 
> first one **load data directly from cache_file,which generated previously. If does not exist, then** and then move to cache_data....
>> (A)  Now, first **check the data existance as cache_data**, if it is there in **empty cache_data, then generate train and test list** for cache_data  and also write operation (dump) to **fill the cache_file.** So, for future purpose our data will be taken from cache_file.<br>
>> (B) But, **if cache_data is already exists, then better to load data** from it,to save time.

Still, in case of confusion!, its better to make a flow diagram on paper ownself. ;) :).

---

In [14]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
# joblib is an enhanced version of pickle that is more efficient for storing NumPy arrays

def extract_BoW_features(words_train, words_test, vocabulary_size=5000,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""
    
    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Fit a vectorizer to training documents and use it to transform them
        # NOTE: Training documents have already been preprocessed and tokenized into words;
        #       pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x
        vectorizer = CountVectorizer(max_features=vocabulary_size)
        features_train = vectorizer.fit_transform(words_train).toarray()

        # Apply the same vectorizer to transform the test documents (ignore unknown words)
        features_test = vectorizer.transform(words_test).toarray()
        
        # NOTE: Remember to convert the features using .toarray() for a compact representation
        
        # Write to cache file for future runs (store vocabulary as well)
        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        features_train, features_test, vocabulary = (cache_data['features_train'],
                cache_data['features_test'], cache_data['vocabulary'])
    
    # Return both the extracted features as well as the vocabulary
    return features_train, features_test, vocabulary

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [15]:
# Extract Bag of Words features for both training and test datasets
train_X, test_X, vocabulary = extract_BoW_features(train_X, test_X)

Wrote features to cache file: bow_features.pkl


In [16]:
len(train_X[100])

5000

## Upload data to S3 [Create DataBase]

### Preparation for classification of XGBoost Algorithm

SageMaker has predefined XGBoost Algirthm for classificatio task. But for better accuracy and avoid overfitting I want to use validation dataset. For that, first we will give first 10000 review to validation and then give data to XGBoost in panda dataframe format. The data is stored in S3. 

In [17]:
import pandas as pd

# Earlier we shuffled the training dataset so to make things simple we can just assign
# the first 10 000 reviews to the validation set and use the remaining reviews for training.
val_X = pd.DataFrame(train_X[:10000])
train_X = pd.DataFrame(train_X[10000:])

val_y = pd.DataFrame(train_y[:10000])
train_y = pd.DataFrame(train_y[10000:])

In [18]:
train_X.head()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,2,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [19]:
val_X.head()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
train_y.head()

,0
0,0
1,1
2,0
3,0
4,0


In [21]:
val_y.head()

,0
0,1
1,1
2,0
3,1
4,1


In [22]:
# First we make sure that the local directory in which we'd like to store the training and validation csv files exists.
data_dir = '../data/sentiment_web_app'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

> The documentation for the XGBoost algorithm of AWS clearly requires data without headings. For future visualization purpose we can add heading after ML operation.

In [23]:
# save data to dictionary (take some time - 1 min) , check your local instance to see the directory
pd.DataFrame(test_X).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)                           # test.csv

pd.concat([val_y, val_X], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)        # validation.csv

pd.concat([train_y, train_X], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)         # train.csv

here, below I am reseting my memory, well, it depends on you instance type. Previousely I do the same work with instance type t2.m2.medium (2GB), but it gave me memory error: "Unable to allocate 954. MiB for an array with shape (25000, 5000) and data type int64" , to avoid it I increase my instance type to ml.m5.large (8GB). It actually increase my cost to 0.26  /ℎ𝑟𝑓𝑟𝑜𝑚0.02 /hr. But, it gives me high internet setup with process speed up. Actually, I will set ml.m4.large for training.

If you are new then please check documentation, you may give some hours of free services, which actually devide in modeling hours, training hours and deploying hours. For me it was 250 hrs, 50 hrs and 125 hours accordingly.

In [24]:
# initialize memory storage (so, set a bit of memory to None).

test_X = train_X = val_X = train_y = val_y = None

Another, important point is, you can not erase such memory every time because when you train deep algorithms, where data will use again and again.

---

## Uploading Training/validation to S3 

### Flow :=> Local_Dir    -TO-   S3(data)    -TO-     Sagemaker(training)    -TO-    S3(result)    -TO-   Local_Dir 

here, Local_Dir is our notebook instance, not your machine space. Check Jupyter Notebook instance main folde, for that.

Here, I am going to use sagemaker's high level functionality so, all the background work will be done by sagemaker ownself, and I just need to provide resources, commands and requirements to sagemaker. This is regid but quicker approach.

There is posibility of Low level functionality, which give us chance to provide flexiblility to model, but when you need to do some research around your result. Well, here in future I will use auto Hyper parameter tuning, to get best answer (with high accuracy) for our dataset problem. So, its nice to use highlevel features.

#### Let's start real work with SAGEMAKER

In [25]:
import sagemaker                                                                # call sagemaker
session = sagemaker.Session()                                                   # create  session for sagemaker 
prefix = 'sentiment-xgboost'                                                    # prefix will be used for unique name identification (in near future)

# set specific location on S3 for easy access 
test_location = session.upload_data(os.path.join(data_dir,'test.csv'),key_prefix= prefix)           # upload test data 
val_location = session.upload_data(os.path.join(data_dir,'validation.csv'),key_prefix = prefix)    # upload validation data
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'),key_prefix = prefix)        # upload train data 

'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.
'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.
'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.


## Create XGBoost model tuning requirement [Development Phase-Model Building]

As I declared before, I am using high level API, helps me to get answer quickly without more flexibility. But, after auto tuing we will get the best answer. Now, here before training, we need to do some setup. 

Sagemaker model creation : it's ecosystem has three different objects, which are interactive with eachother. 
1. Model Artifacts
2. Training Code (container)
3. Inference Code (container)

Model artifact is Model itself. The training code use training data, and create model artifacts. Inference code use the model artifacts to predict new data. 

Sagemaker use docker containers. So, after all docker container is one kind of package of code with proper sequence

In [27]:
from sagemaker import get_execution_role                                          
role = get_execution_role()                                                     # create model execution role = IAM role, for giving permission to specific person or user group (to control unauthorize access)

In [28]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(session.boto_region_name,'xgboost')                   # set container for giving private space to model (when you have more than one deploying model)

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
	get_image_uri(region, 'xgboost', '1.0-1').


In [29]:
# specify model with requried parameters 
xgb = sagemaker.estimator.Estimator(container,                                  # initialize modelxgb = sagemaker.estimator.Estimator(container,                                  # define container (where to take data)
                                    role,                                       # define role (who give permission for this)
                                    train_instance_count = 1,                   # instance will used for task (more instance, more power, more expense)
                                    train_instance_type = 'ml.m4.xlarge',       # power of isntance (more power, more expense, less execution time) , its different than your notebook instance. 
                                    output_path = 's3://{}/{}/output'.format(session.default_bucket(),prefix),   # where to save
                                    sagemaker_session= session)                 # define session (the current one)



xgb.set_hyperparameters(max_depth=5,                                             # set parameters
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=500)


## Fit the XGBoost model [Development -Training Phase]

We already defined model, and also generated the model data. 

Now the next step will be to fit data within model. Means... train our model on dataset. It takes time and for training you have two options in term of instance capacity. 

For me **m1.m4.xlarge** is still in free-tier hours(125hr). So, I am going to use it. otherwise the notebook instance **(m1.m5.xlarge)** which I used is better than this. But, as I discussed earlier **I had problem with cache data of instance memory. So, I used high power model building instance.** You are free to use any. 
> *Please refer the Sagemaker documentation for more information regarding price and capacity. Thank you*

In [30]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

In [31]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})            #[3:23-------------->3:]

2020-06-14 19:23:11 Starting - Starting the training job...
2020-06-14 19:23:13 Starting - Launching requested ML instances.........
2020-06-14 19:24:49 Starting - Preparing the instances for training...
2020-06-14 19:25:39 Downloading - Downloading input data...
2020-06-14 19:26:13 Training - Downloading the training image...
2020-06-14 19:26:33 Training - Training image download completed. Training in progress.Arguments: train
[2020-06-14:19:26:33:INFO] Running standalone xgboost training.
[2020-06-14:19:26:33:INFO] File size need to be processed in the node: 238.5mb. Available memory size in the node: 8474.78mb
[2020-06-14:19:26:33:INFO] Determined delimiter of CSV input is ','
[19:26:33] S3DistributionType set as FullyReplicated
[19:26:35] 15000x5000 matrix with 75000000 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2020-06-14:19:26:35:INFO] Determined delimiter of CSV input is ','
[19:26:35] S3DistributionType set as FullyReplicated
[19:26:36]

[19:27:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 10 pruned nodes, max_depth=5
[39]#011train-error:0.157067#011validation-error:0.1839
[19:27:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 8 pruned nodes, max_depth=5
[40]#011train-error:0.1558#011validation-error:0.1832
[19:27:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=5
[41]#011train-error:0.154667#011validation-error:0.1841
[19:27:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 6 pruned nodes, max_depth=5
[42]#011train-error:0.151333#011validation-error:0.1823
[19:27:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 2 pruned nodes, max_depth=5
[43]#011train-error:0.149733#011validation-error:0.1805
[19:27:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 6 pruned nodes, max_depth=5
[44]#011train-error:0.148333#011validation-error:

[19:28:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 12 pruned nodes, max_depth=5
[87]#011train-error:0.112267#011validation-error:0.157
[19:28:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 8 pruned nodes, max_depth=5
[88]#011train-error:0.111267#011validation-error:0.1572
[19:28:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 10 pruned nodes, max_depth=5
[89]#011train-error:0.11#011validation-error:0.1568
[19:28:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 10 pruned nodes, max_depth=5
[90]#011train-error:0.110067#011validation-error:0.1558
[19:28:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 14 pruned nodes, max_depth=5
[91]#011train-error:0.108667#011validation-error:0.1567
[19:28:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 6 pruned nodes, max_depth=5
[92]#011train-error:0.108#011validation-error:0.1

Now, our model is trained, Check our instance for model info. In case if you have problem to find it, then better option is log_files of sagemaker will give you idea of background procedure.

---
## Testing Model [Testing Phase]

I will use SageMakers Batch Transform functionality.

Batch Transform is a convenient way to perform inference on a large dataset in a way that is not realtime. That is, we don't necessarily need to use our model's results immediately and instead we can peform inference on a large number of samples.

**Applications:**
>Industries, which run their business continueously and want to predict their growth and customer service periodically, may be at the end of week, or end of month. They will use batch transform. So, its not used for realtime applications. Small businesses mostly use it. Sometime industry giants use it for specific problem solution. (as an example, some specific region have issue with specific type of product, then for 5W QA analysis they can use it.) 

---
the following procedure takes some time (5 to 10 minute)...

In [32]:
xgb_transformer = xgb.transformer(instance_count =1, instance_type = 'ml.m4.xlarge')       # used Batch_transform method from sagemaker

xgb_transformer.transform(test_location,content_type = 'text/csv',split_type= 'Line')     # read data from test location for predictinog result.

xgb_transformer.wait()                                                                     # wait for response (to avoid long output format like last one, 
#                                                                                                all information works in background and catched by view_log)

......................Arguments: serve
[2020-06-14 19:33:10 +0000] [1] [INFO] Starting gunicorn 19.7.1
[2020-06-14 19:33:10 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2020-06-14 19:33:10 +0000] [1] [INFO] Using worker: gevent
[2020-06-14 19:33:10 +0000] [38] [INFO] Booting worker with pid: 38
[2020-06-14 19:33:10 +0000] [39] [INFO] Booting worker with pid: 39
[2020-06-14 19:33:10 +0000] [40] [INFO] Booting worker with pid: 40
[2020-06-14 19:33:10 +0000] [41] [INFO] Booting worker with pid: 41
[2020-06-14:19:33:10:INFO] Model loaded successfully for worker : 38
[2020-06-14:19:33:10:INFO] Model loaded successfully for worker : 39
[2020-06-14:19:33:10:INFO] Model loaded successfully for worker : 40
[2020-06-14:19:33:10:INFO] Model loaded successfully for worker : 41
2020-06-14T19:33:40.381:[sagemaker logs]: MaxConcurrentTransforms=4, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
[2020-06-14:19:33:43:INFO] Sniff delimiter as ','
[2020-06-14:19:33:43:INFO] Sniff delimiter as ',


[2020-06-14:19:34:03:INFO] Sniff delimiter as ','
[2020-06-14:19:34:03:INFO] Determined delimiter of CSV input is ','
[2020-06-14:19:34:03:INFO] Sniff delimiter as ','
[2020-06-14:19:34:03:INFO] Determined delimiter of CSV input is ','
[2020-06-14:19:34:05:INFO] Sniff delimiter as ','
[2020-06-14:19:34:05:INFO] Determined delimiter of CSV input is ','
[2020-06-14:19:34:05:INFO] Sniff delimiter as ','
[2020-06-14:19:34:05:INFO] Determined delimiter of CSV input is ','
[2020-06-14:19:34:05:INFO] Sniff delimiter as ','
[2020-06-14:19:34:05:INFO] Determined delimiter of CSV input is ','
[2020-06-14:19:34:05:INFO] Sniff delimiter as ','
[2020-06-14:19:34:05:INFO] Determined delimiter of CSV input is ','
[2020-06-14:19:34:05:INFO] Sniff delimiter as ','
[2020-06-14:19:34:05:INFO] Sniff delimiter as ','
[2020-06-14:19:34:05:INFO] Determined delimiter of CSV input is ','
[2020-06-14:19:34:06:INFO] Sniff delimiter as ','
[2020-06-14:19:34:06:INFO] Determined delimiter of CSV input is ','
[2020

Now, the transform job has executed and the result, the estimated sentiment of each review, has been saved on S3. Since we would rather work on this file locally we can perform a bit of notebook magic to copy the file to the `data_dir`. (to local notebook instance)

In [33]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

download: s3://sagemaker-us-west-2-337299574287/xgboost-2020-06-14-19-29-44-762/test.csv.out to ../data/sentiment_web_app/test.csv.out


The last step is now to read in the output from our model, convert the output to something a little more usable, in this case we want the sentiment to be either `1` (positive) or `0` (negative), and then compare to the ground truth labels.

In [34]:
# For, accuracy metric calculation. 
predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.8464

## Step 6: Deploying the model [Deployment Phase]

Once we construct and fit our model, SageMaker stores the resulting model artifacts and we can use those to deploy an endpoint (inference code). To see this, look in the SageMaker console and you should see that a model has been created along with a link to the S3 location where the model artifacts have been stored.

Deploying an endpoint is a lot like training the model with a few important differences. The first is that a deployed model doesn't change the model artifacts, so as you send it various testing instances the model won't change. Another difference is that since we aren't performing a fixed computation, as we were in the training step or while performing a batch transform, the compute instance that gets started stays running until we tell it to stop. This is important to note as if we forget and leave it running we will be charged the entire time.

In other words **If you are no longer using a deployed endpoint, shut it down!**

In [36]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')   

----------------!

### Testing the model (again with deployed model)

OUr model is already deployed, but we need to check it before using it form endpoint. For tha I will test the model form newly deployed endpoint. This thing ensure us about two things. 
1. Model Testing
2. Endpoint permission

But, now we send our data from endpoint, not from notebook instance (sagemaker, S3) so, we can not send huge information, the better way is to divide them in chunk. Data must be serialized for model.

Let's satisfy above requirement...

In [41]:
from sagemaker.predictor import csv_serializer

# We need to tell the endpoint what format the data we are sending is in so that SageMaker can perform the serialization.
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

In [42]:
# We split the data into chunks and send each chunk seperately, accumulating the results.
           
def predict(data, rows=512):                                                           # max limit of data 512 row. 
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])
    
    return np.fromstring(predictions[1:], sep=',')

In [43]:
test_X = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None).values

predictions = predict(test_X)
predictions = [round(num) for num in predictions]

Lastly, we check to see what the accuracy of our model is.

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.8464

And the results here should agree with the model testing that we did earlier using the batch transform job. Notice that here, all the testing and predicting jobs executed in background. You can check it from log file of sagemaker. 

### Cleaning up

Now that we've determined that deploying our model works as expected, we are going to shut it down. Remember that the longer the endpoint is left running, the greater the cost and since we have a bit more work to do before we are able to use our endpoint with our simple web app, we should shut everything down.

In [66]:
xgb_predictor.delete_endpoint()

## Release Model: 

As I discussed before, our main purpose is to deploy app in open environment so we can use it from web api. But, currently whole procedure work through Sagemaker API. so, first of all Its better to deploy with some medium which can allow our endpoing from open permission. For that Lambda is used. But, before that we need o authenticate our web app with AWS. 

we will create one new endpoint (proxy-endpoint), which will be public and no need to authenticate from AWS-sagemaker. Then main question arise is about the data processing. Because, when we deploy our model, we never think about the data preprocessing step. When we give our data it need to be serialized and convert as bag of words. 

Moreover, we can not give this load to our web application itself. So, Its better to use AWS-Lambda function. The simple process is as follow.



<img src="Web App Diagram.svg">

What this means is that when someone uses our web app, the following will occur.

1. To begin with, a user will type out a review and enter it into our web app.

2. Then, our web app will send that review to an endpoint that we created using API Gateway. This endpoint will be constructed so that anyone (including our web app) can use it.

3. API Gateway will forward the data on to the Lambda function

4. Once the Lambda function receives the user's review, it will process that review by tokenizing it and then creating a bag of words encoding of the result. After that, it will send the processed review off to our deployed model.

5. Once the deployed model performs inference on the processed review, the resulting sentiment will be returned back to the Lambda function.

6. Our Lambda function will then return the sentiment result back to our web app using the endpoint that was constructed using API Gateway.

---


## Processing a single review

For now, suppose we are given a movie review by our user in the form of a string, like so:

In [67]:
test_review = "Nothing but a disgusting materialistic pageant of glistening abed remote control greed zombies, totally devoid of any heart or heat. A romantic comedy that has zero romantic chemestry and zero laughs!"

How do we go from this string to the bag of words feature vector that is expected by our model?

If we recall at the beginning of this notebook, the first step is to remove any unnecessary characters using the `review_to_words` method. Remember that we intentionally did this in a very simplistic way. This is because we are going to have to copy this method to our (eventual) Lambda function (we will go into more detail later) and this means it needs to be rather simplistic.

In [68]:
test_words = review_to_words(test_review)
print(test_words)

nothing but a disgusting materialistic pageant of glistening abed remote control greed zombies totally devoid of any heart or heat a romantic comedy that has zero romantic chemestry and zero laughs


Next, we need to construct a bag of words embedding of the `test_words` string. To do this, remember that a bag of words embedding uses a `vocabulary` consisting of the most frequently appearing words in a set of documents. Then, for each word in the vocabulary we record the number of times that word appears in `test_words`. We constructed the `vocabulary` earlier using the training set for our problem so encoding `test_words` is relatively straightforward.

In [69]:
def bow_encoding(words, vocabulary):                    # our data preprocessing function 
    bow = [0] * len(vocabulary) # Start by setting the count for each word in the vocabulary to zero.
    for word in words.split():  # For each word in the string
        if word in vocabulary:  # If the word is one that occurs in the vocabulary, increase its count.
            bow[vocabulary[word]] += 1
    return bow

In [70]:
test_bow = bow_encoding(test_words, vocabulary)          
print(test_bow)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

The output is in 0 and 1, but main problem is regarding format. Its not serialized and not even in required format for our mode. I mean not csv or text format. So, first we need to change it.

In [71]:
len(test_bow)                          # Note: this are words not rows, so please do not confuse with 512 chunk size                            

5000

So now we know how to construct a bag of words encoding of a user provided review, how to we send it to our endpoint? First, we need to start the endpoint back up.

In [72]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')      # redeploy model to regenerate endpoint!

-------------!

This is the same thing that, we did earlier when we tested our deployed model and send `test_bow` to our endpoint using the `xgb_predictor` object. But, when we have lambda in deployment, then we can not access this endpoint directly.

The common approach is to use boto3 library from AWS. This will provide an API which runs with AWS service and will give us leverage to work with lambda, sagemaker and other functions. 


In [73]:
import boto3

runtime = boto3.Session().client('sagemaker-runtime')                 # handle runtime of sagemaker.

And now that we have access to the SageMaker runtime, we can ask it to make use of (invoke) an endpoint that has already been created. However, we need to provide SageMaker with the name of the deployed endpoint. To find this out we can print it out using the `xgb_predictor` object.

In [74]:
xgb_predictor.endpoint             # 'xgboost-2020-06-14-19-23-11-456'

'xgboost-2020-06-14-19-23-11-456'

Using the SageMaker runtime and the name of our endpoint, we can invoke the endpoint and send it the `test_bow` data. Note that the invoke_endpoint works with live applications. So, our model will not run continuously. It invoke only by user only. So, we do not need to give continuous charge. ---- << ADD >> 

In [75]:
response = runtime.invoke_endpoint(EndpointName = xgb_predictor.endpoint, # The name of the endpoint we created
                                       ContentType = 'text/csv',                     # The data format that is expected
                                       Body = test_bow)

ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Here, I got error, because of incompatible format. We need serialized input with "csv/text" format. As we defined. 

In [76]:
response = runtime.invoke_endpoint(EndpointName = xgb_predictor.endpoint, # The name of the endpoint we created
                                       ContentType = 'text/csv',                     # The data format that is expected
                                       Body = ','.join([str(val) for val in test_bow]).encode('utf-8'))

In [77]:
print(response)

{'ResponseMetadata': {'RequestId': '5f08de00-17ce-4bbe-8089-3b9dff235b3b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5f08de00-17ce-4bbe-8089-3b9dff235b3b', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Sun, 14 Jun 2020 22:55:20 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '14'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f34a684ca58>}


As we can see, the response from our model is a somewhat complicated looking dict that contains a bunch of information. The bit that we are most interested in is `'Body'` object which is a streaming object that we need to `read` in order to make use of.

In [78]:
response = response['Body'].read().decode('utf-8')           # 0.5 is breaking point for our app, so >50 will positive and <0.50 will negative.
print(response)

0.500335335732


Now that we know how to process the incoming user data we can start setting up the infrastructure to make our simple web app work. To do this we will make use of two different services. Amazon's Lambda and API Gateway services.

Lambda is a service which allows someone to write some relatively simple code and have it executed whenever a chosen trigger occurs. For example, you may want to update a database whenever new data is uploaded to a folder stored on S3.

API Gateway is a service that allows you to create HTTP endpoints (url addresses) which are connected to other AWS services. One of the benefits to this is that you get to decide what credentials, if any, are required to access these endpoints.

In our case we are going to set up an HTTP endpoint through API Gateway which is open to the public. Then, whenever anyone sends data to our public endpoint we will trigger a Lambda function which will send the input (in our case a review) to our model's endpoint and then return the result.

### Setting up a Lambda function

The first thing we are going to do is set up a Lambda function. This Lambda function will be executed whenever our public API has data sent to it. When it is executed it will receive the data, perform any sort of processing that is required, send the data (the review) to the SageMaker endpoint we've created and then return the result.

#### Part A: Create an IAM Role for the Lambda function

Since we want the Lambda function to call a SageMaker endpoint, we need to make sure that it has permission to do so. To do this, we will construct a role that we can later give the Lambda function.

Using the AWS Console, navigate to the **IAM** page and click on **Roles**. Then, click on **Create role**. Make sure that the **AWS service** is the type of trusted entity selected and choose **Lambda** as the service that will use this role, then click **Next: Permissions**.

In the search box type `sagemaker` and select the check box next to the **AmazonSageMakerFullAccess** policy. Then, click on **Next: Review**.

Lastly, give this role a name. Make sure you use a name that you will remember later on, for example `LambdaSageMakerRole`. Then, click on **Create role**.

#### Part B: Create a Lambda function

Now it is time to actually create the Lambda function. Remember from earlier that in order to process the user provided input and send it to our endpoint we need to gather two pieces of information:

 - The name of the endpoint, and
 - the vocabulary object.

We will copy these pieces of information to our Lambda function after we create it.

To start, using the AWS Console, navigate to the AWS Lambda page and click on **Create a function**. When you get to the next page, make sure that **Author from scratch** is selected. Now, name your Lambda function, using a name that you will remember later on, for example `sentiment_analysis_xgboost_func`. Make sure that the **Python 3.6** runtime is selected and then choose the role that you created in the previous part. Then, click on **Create Function**.

On the next page you will see some information about the Lambda function you've just created. If you scroll down you should see an editor in which you can write the code that will be executed when your Lambda function is triggered. Collecting the code we wrote above to process a single review and adding it to the provided example `lambda_handler` we arrive at the following.

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

# And we need the regular expression library to do some of the data processing
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def review_to_words(review):
    words = REPLACE_NO_SPACE.sub("", review.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    return words
    
def bow_encoding(words, vocabulary):
    bow = [0] * len(vocabulary) # Start by setting the count for each word in the vocabulary to zero.
    for word in words.split():  # For each word in the string
        if word in vocabulary:  # If the word is one that occurs in the vocabulary, increase its count.
            bow[vocabulary[word]] += 1
    return bow


def lambda_handler(event, context):
    
    vocab = "*** ACTUAL VOCABULARY GOES HERE ***"
    
    words = review_to_words(event['body'])
    bow = bow_encoding(words, vocab)

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '***ENDPOINT NAME HERE***',# The name of the endpoint we created
                                       ContentType = 'text/csv',                 # The data format that is expected
                                       Body = ','.join([str(val) for val in bow]).encode('utf-8')) # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')
    
    # Round the result so that our web app only gets '1' or '0' as a response.
    result = round(float(result))

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : str(result)
    }
```

Once you have copy and pasted the code above into the Lambda code editor, replace the `**ENDPOINT NAME HERE**` portion with the name of the endpoint that we deployed earlier. You can determine the name of the endpoint using the code cell below.

In [79]:
xgb_predictor.endpoint           # yout endpoint

'xgboost-2020-06-14-19-23-11-456'

In addition, you will need to copy the vocabulary dict to the appropriate place in the code at the beginning of the `lambda_handler` method. The cell below prints out the vocabulary dict in a way that is easy to copy and paste.

In [80]:
print(str(vocabulary))

{'water': 4816, 'is': 2360, 'well': 4843, 'made': 2697, 'first': 1736, 'film': 1716, 'from': 1840, 'france': 1816, 'about': 67, 'young': 4989, 'female': 1693, 'sexuality': 3909, 'and': 226, 'friendship': 1838, 'works': 4945, 'with': 4911, 'slightly': 4029, 'material': 2766, 'that': 4424, 'as': 311, 'off': 3084, 'putting': 3482, 'to': 4507, 'some': 4061, 'it': 2366, 'others': 3136, 'the': 4426, 'focuses': 1768, 'on': 3105, 'three': 4469, 'middle': 2833, 'class': 833, 'teenage': 4389, 'girls': 1911, 'in': 2257, 'new': 3003, 'paris': 3189, 'their': 4431, 'lives': 2628, 'around': 297, 'big': 478, 'swimming': 4338, 'pool': 3327, 'where': 4864, 'two': 4620, 'of': 3083, 'are': 285, 'part': 3193, 'ballet': 389, 'team': 4378, 'such': 4268, 'parents': 3188, 'work': 4940, 'school': 3813, 'have': 2052, 'been': 438, 'central': 735, 'belongs': 460, 'but': 636, 'marie': 2737, 'who': 4871, 'not': 3040, 'thinks': 4454, 'she': 3923, 'would': 4957, 'like': 2599, 'be': 418, 'an': 223, 'blonde': 511, 'whom

Once you have added the endpoint name to the Lambda function, click on **Save**. Your Lambda function is now up and running. Next we need to create a way for our web app to execute the Lambda function.

### Setting up API Gateway

Now that our Lambda function is set up, it is time to create a new API using API Gateway that will trigger the Lambda function we have just created.

Using AWS Console, navigate to **Amazon API Gateway** and then click on **Get started**.

On the next page, make sure that **New API** is selected and give the new api a name, for example, `sentiment_analysis_web_app`. Then, click on **Create API**.

Now we have created an API, however it doesn't currently do anything. What we want it to do is to trigger the Lambda function that we created earlier.

Select the **Actions** dropdown menu and click **Create Method**. A new blank method will be created, select its dropdown menu and select **POST**, then click on the check mark beside it.

For the integration point, make sure that **Lambda Function** is selected and click on the **Use Lambda Proxy integration**. This option makes sure that the data that is sent to the API is then sent directly to the Lambda function with no processing. It also means that the return value must be a proper response object as it will also not be processed by API Gateway.

Type the name of the Lambda function you created earlier into the **Lambda Function** text entry box and then click on **Save**. Click on **OK** in the pop-up box that then appears, giving permission to API Gateway to invoke the Lambda function you created.

The last step in creating the API Gateway is to select the **Actions** dropdown and click on **Deploy API**. You will need to create a new Deployment stage and name it anything you like, for example `prod`.

You have now successfully set up a public API to access your SageMaker model. Make sure to copy or write down the URL provided to invoke your newly created public API as this will be needed in the next step. This URL can be found at the top of the page, highlighted in blue next to the text **Invoke URL**.

## Step 7: Deploying our web app

Now that we have a publicly available API, we can start using it in a web app. For our purposes, we have provided a simple static html file which can make use of the public api you created earlier.

In the `website` folder there should be a file called `index.html`. Download the file to your computer and open that file up in a text editor of your choice. There should be a line which contains **\*\*REPLACE WITH PUBLIC API URL\*\***. Replace this string with the url that you wrote down in the last step and then save the file.

Now, if you open `index.html` on your local computer, your browser will behave as a local web server and you can use the provided site to interact with your SageMaker model.

If you'd like to go further, you can host this html file anywhere you'd like, for example using github or hosting a static site on Amazon's S3. Once you have done this you can share the link with anyone you'd like and have them play with it too!

> **Important Note** In order for the web app to communicate with the SageMaker endpoint, the endpoint has to actually be deployed and running. This means that you are paying for it. Make sure that the endpoint is running when you want to use the web app but that you shut it down when you don't need it, otherwise you will end up with a surprisingly large AWS bill.

### Delete the endpoint

Remember to always shut down your endpoint if you are no longer using it. You are charged for the length of time that the endpoint is running so if you forget and leave it on you could end up with an unexpectedly large bill.

In [ ]:
xgb_predictor.delete_endpoint()

### Clean up disk and dir (free memory for next prediction)

The default notebook instance on SageMaker doesn't have a lot of excess disk space available. As you continue to complete and execute other notebooks you will eventually fill up this disk space, leading to errors which can be difficult to diagnose. 

Once you are completely finished using a notebook it is a good idea to remove the files that you created along the way. Of course, you can do this from the terminal or from the notebook hub if you would like. The cell below contains some commands to clean up the created files from within the notebook.

In [ ]:
# First we will remove all of the files contained in the data_dir directory
!rm $data_dir/*

# And then we delete the directory itself
!rmdir $data_dir

# Similarly we remove the files in the cache_dir directory and the directory itself
!rm $cache_dir/*
!rmdir $cache_dir